In [4]:
pip install transformers torch

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load pre-trained model and tokenizer
model = AutoModelForCausalLM.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# Example dataset (simplified for demonstration)
dataset = [
    {"prompt": "Write a story about a cat.", "preferred_response": "Once upon a time, there was a cat named Whiskers.", "dispreferred_response": "Cats are not interesting."},
    {"prompt": "Describe a sunny day.", "preferred_response": "The sun was shining brightly in the clear blue sky.", "dispreferred_response": "It was cloudy and rainy."},
    # Add more data points as needed
]

In [7]:
def dpo_loss(model, batch, beta=0.1):
    # Tokenize inputs
    inputs = tokenizer(batch['prompt'], return_tensors='pt')
    preferred = tokenizer(batch['preferred_response'], return_tensors='pt')
    dispreferred = tokenizer(batch['dispreferred_response'], return_tensors='pt')

    # Get model outputs
    preferred_logits = model(**inputs, labels=preferred['input_ids']).logits
    dispreferred_logits = model(**inputs, labels=dispreferred['input_ids']).logits

    # Compute BCE loss
    loss_fn = torch.nn.BCEWithLogitsLoss()
    loss = loss_fn(preferred_logits, torch.ones_like(preferred_logits)) + loss_fn(dispreferred_logits, torch.zeros_like(dispreferred_logits))

    # Regularization term
    regularization = beta * (preferred_logits - dispreferred_logits).mean()

    return loss + regularization

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load pre-trained model and tokenizer
model = AutoModelForCausalLM.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Example dataset (simplified for demonstration)
dataset = [
    {"prompt": "Write a story about a cat.", "preferred_response": "Once upon a time, there was a cat named Whiskers.", "dispreferred_response": "Cats are not interesting."},
    {"prompt": "Describe a sunny day.", "preferred_response": "The sun was shining brightly in the clear blue sky.", "dispreferred_response": "It was cloudy and rainy."},
    # Add more data points as needed
]

def dpo_loss(model, batch, beta=0.1):
    # Get model outputs
    preferred_logits = model(**batch['preferred_inputs'], labels=batch['preferred_labels']).logits
    dispreferred_logits = model(**batch['dispreferred_inputs'], labels=batch['dispreferred_labels']).logits

    # Compute BCE loss
    loss_fn = torch.nn.BCEWithLogitsLoss()
    loss = loss_fn(preferred_logits, torch.ones_like(preferred_logits)) + loss_fn(dispreferred_logits, torch.zeros_like(dispreferred_logits))

    # Regularization term
    regularization = beta * (preferred_logits.mean() - dispreferred_logits.mean())

    return loss + regularization
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Tokenize dataset
tokenized_dataset = []
for batch in dataset:
    # Tokenize inputs
    preferred_inputs = tokenizer(batch['preferred_response'], return_tensors='pt')
    dispreferred_inputs = tokenizer(batch['dispreferred_response'], return_tensors='pt')
    prompt_inputs = tokenizer(batch['prompt'], return_tensors='pt')

    # Create labels (assuming the preferred response is the correct one)
    preferred_labels = preferred_inputs['input_ids'].clone()
    dispreferred_labels = dispreferred_inputs['input_ids'].clone()

    # Move tensors to device
    preferred_inputs = {k: v.to(device) for k, v in preferred_inputs.items()}
    dispreferred_inputs = {k: v.to(device) for k, v in dispreferred_inputs.items()}
    preferred_labels = preferred_labels.to(device)
    dispreferred_labels = dispreferred_labels.to(device)

    # Create batch
    batch = {
        'preferred_inputs': preferred_inputs,
        'dispreferred_inputs': dispreferred_inputs,
        'preferred_labels': preferred_labels,
        'dispreferred_labels': dispreferred_labels,
    }

    tokenized_dataset.append(batch)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Train loop
for epoch in range(5):
    for batch in tokenized_dataset:
        # Compute loss
        loss = dpo_loss(model, batch)

        # Backpropagate and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Epoch 1, Loss: 93.1196517944336
Epoch 1, Loss: 101.7238540649414
Epoch 2, Loss: 85.02855682373047
Epoch 2, Loss: 94.56044006347656
Epoch 3, Loss: 88.35965728759766
Epoch 3, Loss: 87.6749038696289
Epoch 4, Loss: 83.44696807861328
Epoch 4, Loss: 80.8504867553711
Epoch 5, Loss: 76.1731948852539
Epoch 5, Loss: 74.07881927490234
